In [0]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm
from PIL import Image #import Python Image Library
import numpy as np
import os
import glob
import re
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import regularizers
from keras.utils import np_utils
from keras import backend as K

In [0]:
DATADIR = '/content/drive/My Drive/UCMerced_LandUse/Images/'

CATEGORIES = [ 'agricultural',  'airplane',    'baseballdiamond', 'beach',   'buildings',          'chaparral',         'denseresidential',
                'forest',        'freeway',     'golfcourse',      'harbor',  'intersection',       'mediumresidential', 'mobilehomepark',
                'overpass',      'parkinglot',  'river',           'runway',  'sparseresidential',  'storagetanks',      'tenniscourt' ]


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
dirs = os.listdir('/content/drive/My Drive/ucmerced/UCMerced_LandUse/Images/')

In [0]:
img_rows = 256
img_cols = 256
num_class = 21
path = os.path.abspath('.cnn.py') #absolute path of program
path = re.sub('[a-zA-Z\s._]+$', '', path) #remove unintended file
X = []
Y = []
dirs = os.listdir('/content/drive/My Drive/ucmerced/UCMerced_LandUse/Images/')
#dirs=dirs[:-1]
print(len(dirs))
'''

'''

In [0]:
label = 0
for i in dirs:
	n = 0
	count = 0
	for pic in glob.glob('/content/drive/My Drive/ucmerced/UCMerced_LandUse/Images/'+i+'/*.tif'):
		im = Image.open(pic)
		im = np.array(im)
		if((im.shape[0]==256) and (im.shape[1] ==256) and n<90): #get only 90 data
			r = im[:,:,0]
			g = im[:,:,1]
			b = im[:,:,2]
			X.append([r,g,b])
			Y.append([label])
			n = n+1
	print(n)
	label = label + 1

In [0]:
import numpy as np
x_bck=X
y_bck=Y


X =  np.array(X)
Y =  np.array(Y)
X = X.reshape(X.shape[0], img_rows, img_cols, 3)
X.shape


In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

input_shape = (img_rows, img_cols, 3)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
y_train = keras.utils.to_categorical(y_train, 21)
y_test = keras.utils.to_categorical(y_test, 21)

In [0]:
import time 
import cv2 
from scipy.fftpack import dct 

a_256=[]
b_256=[]
#y = np.random.random((5000, 6, 6, 3))
t1 = time.time()
a_256 = np.zeros((len(X_train), 256, 256, 3))
b_256 = np.zeros((len(X_test), 256, 256, 3))
for i in range(len(X_train)):
    for j in range(3):
        a_256[i, : , : , j] = cv2.dct(X_train[i, : , : , j])
        np.transpose(a_256, [0, 2, 1, 3])

for i in range(len(X_train)):
    for j in range(3):
        a_256[i, :, :, j] = cv2.dct(X_train[i, :, :, j]) 

for i in range(0,len(X_test)):
    for j in range(3):
        b_256[i, : , : , j] = cv2.dct(X_test[i, : , : , j])
        np.transpose(b_256, [0, 2, 1, 3])

for i in range(len(X_test)):
    for j in range(3):
        b_256[i, :, :, j] = cv2.dct(X_test[i, :, :, j]) 
 


In [0]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(64,64,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(21, activation='softmax'))
model.compile(loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])

##New model

In [0]:
model_big = []

In [0]:
model_big = Sequential()
model_big.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_shape))
model_big.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(1e-5)))
model_big.add(MaxPooling2D(pool_size=(2,2), strides=2))
model_big.add(Dropout(0.25))
model_big.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(1e-5)))
model_big.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(1e-5)))
model_big.add(MaxPooling2D(pool_size=(2,2), strides=2))
model_big.add(Dropout(0.25))
model_big.add(Conv2D(128, (3,3), activation='relu', kernel_regularizer=regularizers.l2(1e-5)))
model_big.add(MaxPooling2D(pool_size=(2,2), strides=2))
model_big.add(Flatten())
model_big.add(Dense(128, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4), bias_regularizer=regularizers.l2(1e-4), activity_regularizer=regularizers.l2(1e-5)))
model_big.add(Dropout(0.5))
model_big.add(Dense(21, activation='softmax'))
model_big.compile(loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])

In [0]:
model_big.fit(a_256, y_train, batch_size=50, nb_epoch=150, verbose=1, validation_data=(b_256, y_test))

In [0]:
loss, acc = model.evaluate(b_256, y_test, verbose=0)
print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

#Function for cutting the DCT output

In [0]:
def cut_dct(cut):
    a_cut = a_256[:,:cut,:cut,:]
    b_cut = b_256[:,:cut,:cut,:]
    model_big.fit(a_cut, y_train, batch_size=50, nb_epoch=100, verbose=1, validation_data=(b_cut, y_test))
    loss, acc = model_big.evaluate(b_cut, y_test, verbose=0)
    print('\n\n\n')
    print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

#### Change Input Size of model according to the cut

In [0]:
cut_dct(128)

In [0]:
cut_dct(64)

In [0]:
cut_dct(32)

In [0]:
cut_dct(16)